# KOSIS 준공 후 미분양 현황 데이터 수집 및 분석

이 노트북은 PublicDataReader 라이브러리를 사용하여 국토교통부의 시군구별 공사완료 후 미분양 현황 데이터를 수집하고 분석합니다.

**데이터 범위:**
- 기간: 2010년 1월 ~ 2025년 12월 (최신월까지)
- 지역: 전국 시도별 + 시군구별 (216개 시군구)
- 부문: 전체 (공공+민간 합계)
- 규모: 전체 (모든 면적 합계)

## 1. 라이브러리 Import

In [ ]:
from PublicDataReader import Kosis
import pandas as pd
from datetime import datetime

# 그래프 라이브러리 (Plotly 사용)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plotly 기본 설정
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✓ 라이브러리 Import 완료")
print("✓ Plotly 인터랙티브 그래프 사용")

## 2. 데이터 수집 함수 정의

In [ ]:
# 준공_후_미분양.py에서 collect_completed_unsold_data 함수를 그대로 가져옴
exec(open('준공_후_미분양.py', encoding='utf-8').read())

## 3. 데이터 수집 실행 (선택사항)

**주의:** 이 셀을 실행하면 KOSIS API로부터 데이터를 새로 수집합니다 (약 3-5분 소요).  
이미 수집된 CSV 파일이 있다면 다음 셀에서 파일을 직접 로드하세요.

In [ ]:
# 새로 데이터 수집
df_raw = collect_completed_unsold_data()

if df_raw is not None:
    # 원본 데이터 저장
    current_date = datetime.now().strftime("%Y%m%d")
    df_raw.to_csv(f"준공_후_미분양_시군구별_{current_date}.csv", index=False, encoding='utf-8-sig')
    print(f"원본 데이터가 '준공_후_미분양_시군구별_{current_date}.csv' 파일로 저장되었습니다.")

## 4. 기존 데이터 로드 (선택사항)

이미 수집된 CSV 파일을 로드합니다.

In [ ]:
# 기존 원본 데이터 로드
df_raw = pd.read_csv('준공_후_미분양_시군구별_20251227.csv', encoding='utf-8-sig')
print(f"원본 데이터 로드 완료: {len(df_raw):,}건")
print(f"수집 기간: {df_raw['수록시점'].min()} ~ {df_raw['수록시점'].max()}")

## 5. 최종 피벗 테이블 생성

In [ ]:
# 최종 피벗 테이블 생성
df_final = create_final_pivot_table(df_raw)

# 데이터 확인
print("\n=== 최종 데이터 미리보기 ===")
display(df_final.head(20))

print("\n=== 데이터 통계 ===")
print(f"총 행 수: {len(df_final):,}")
print(f"시점 범위: {df_final['시점'].min()} ~ {df_final['시점'].max()}")
print(f"시도 개수: {len(df_final['시도'].unique())}개")
print(f"시군구 개수: {len(df_final['시군구'].unique())}개")
print(f"시도 목록: {', '.join(sorted(df_final['시도'].unique()))}")

## 6. 데이터 탐색

In [ ]:
# 특정 시도의 데이터 확인
sido_name = '서울특별시'
df_seoul = df_final[df_final['시도'] == sido_name]

print(f"\n=== {sido_name} 데이터 ===")
display(df_seoul.head(20))

# 시군구별 총 미분양 (2010-2025)
print(f"\n=== {sido_name} 시군구별 총 미분양 수 (2010-2025) ===")
sigungu_total = df_seoul.groupby('시군구')['미분양수'].sum().sort_values(ascending=False)
display(sigungu_total.head(10))

## 7. 시각화

### 7.1. 전국 준공 후 미분양 추이

In [ ]:
# 시점을 datetime으로 변환
df_final['시점_dt'] = pd.to_datetime(df_final['시점'], format='%Y.%m')

# 전국 월별 미분양 합계
df_monthly_total = df_final.groupby('시점_dt')['미분양수'].sum().reset_index()

# 그래프 생성
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_monthly_total['시점_dt'],
        y=df_monthly_total['미분양수'],
        mode='lines+markers',
        name='전국 준공 후 미분양',
        line=dict(width=2, color='#9B59B6'),
        marker=dict(size=4),
        hovertemplate='%{x|%Y.%m}<br>미분양: %{y:,.0f}호<extra></extra>'
    )
)

# X축 설정
fig.update_xaxes(
    title_text='시점',
    tickangle=-90,
    dtick="M12"
)

# Y축 설정
fig.update_yaxes(
    title_text='미분양 수 (호)',
    tickformat=","
)

# 레이아웃 설정
fig.update_layout(
    title='전국 준공 후 미분양 추이 (2010-2025)',
    title_font_size=16,
    height=500,
    hovermode='x unified',
    font=dict(size=12)
)

fig.show()

### 7.2. 시도별 준공 후 미분양 비교 (최근 1년)

In [ ]:
# 최근 1년 데이터 필터링
latest_date = df_final['시점_dt'].max()
one_year_ago = latest_date - pd.DateOffset(months=12)
df_recent = df_final[df_final['시점_dt'] > one_year_ago]

# 시도별 합계 계산
df_sido_total = df_recent.groupby('시도')['미분양수'].sum().reset_index()
df_sido_total = df_sido_total.sort_values('미분양수', ascending=True)

# 가로 막대 그래프
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_sido_total['미분양수'],
    y=df_sido_total['시도'],
    orientation='h',
    marker=dict(color='#9B59B6'),
    text=df_sido_total['미분양수'].apply(lambda x: f'{x:,.0f}'),
    textposition='inside',
    textfont=dict(color='white'),
    hovertemplate='%{y}<br>미분양: %{x:,.0f}호<extra></extra>'
))

# X축 설정
fig.update_xaxes(tickformat=",")

# 레이아웃 설정
fig.update_layout(
    title=f'시도별 준공 후 미분양 현황 (최근 1년: {one_year_ago.strftime("%Y.%m")} ~ {latest_date.strftime("%Y.%m")})',
    xaxis_title='미분양 수 (호)',
    yaxis_title='시도',
    height=600,
    font=dict(size=12)
)

fig.show()

### 7.3. 시군구별 준공 후 미분양 Top 20 (최근 1개월)

In [ ]:
# 최근 1개월 데이터
df_latest_month = df_final[df_final['시점_dt'] == latest_date]

# Top 20
df_sigungu_top = df_latest_month.nlargest(20, '미분양수')
df_sigungu_top = df_sigungu_top.sort_values('미분양수', ascending=True)

# 시도+시군구 합치기
df_sigungu_top['지역'] = df_sigungu_top['시도'].str.replace('특별시|광역시|특별자치시|특별자치도|도', '', regex=True) + ' ' + df_sigungu_top['시군구']

# 가로 막대 그래프
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_sigungu_top['미분양수'],
    y=df_sigungu_top['지역'],
    orientation='h',
    marker=dict(color=df_sigungu_top['미분양수'], colorscale='Purples'),
    text=df_sigungu_top['미분양수'].apply(lambda x: f'{x:,.0f}'),
    textposition='inside',
    textfont=dict(color='white'),
    hovertemplate='%{y}<br>미분양: %{x:,.0f}호<extra></extra>'
))

# X축 설정
fig.update_xaxes(tickformat=",")

# 레이아웃 설정
fig.update_layout(
    title=f'시군구별 준공 후 미분양 Top 20 ({latest_date.strftime("%Y.%m")})',
    xaxis_title='미분양 수 (호)',
    yaxis_title='지역',
    height=700,
    font=dict(size=11),
    showlegend=False
)

fig.show()

### 7.4. 시도별 준공 후 미분양 추이 비교

In [ ]:
# 시도별 월별 합계
df_sido_time = df_final.groupby(['시점_dt', '시도'])['미분양수'].sum().reset_index()

# 그래프 생성
fig = px.line(
    df_sido_time,
    x='시점_dt',
    y='미분양수',
    color='시도',
    title='시도별 준공 후 미분양 추이 비교 (2010-2025)',
    labels={'시점_dt': '시점', '미분양수': '미분양 수 (호)'},
    height=600
)

# 레이아웃 설정
fig.update_layout(
    hovermode='x unified',
    font=dict(size=12),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

# X축 설정
fig.update_xaxes(tickangle=-90)

# Y축 설정
fig.update_yaxes(tickformat=",")

fig.show()

## 8. 추가 분석 영역

아래 셀에 추가 분석 및 시각화 코드를 작성하세요.

In [ ]:
# 추가 분석 코드를 여기에 작성하세요
